In [1]:
import random as rd
import numpy as np

from glob import glob
from IPython.display import clear_output

In [2]:
def euclid_extended(a, b):
    '''
    Realiza o cálculo do mdc
    entre a e b, retornando
    o mdc e x e y tais que
    ax + by = mdc(a, b).
    '''
    inverted = False
    if b > a:
        a, b = b, a
        inverted = True
        
    
    table = np.array([[a, b], [1, 0], [0, 1]])
    iteration = 0
    while table[0, (iteration + 1) % 2] != 0:
        a, b = table[0, iteration % 2], table[0, (iteration + 1) % 2]
        q = a // b
        r = a % b
        table[:, iteration % 2] -= table[:, (iteration + 1) % 2] * q
        iteration += 1
        
    lcd, x, y = table[:, iteration % 2]
    
    if inverted:
        return lcd, y, x
    else:
        return lcd, x, y

def is_prime(n):
    '''
    Recebe um número e verifica
    se ele é primo
    '''
    if n == 2:
        return True
    
    if n < 2 or n % 2 == 0:
        return False
    
    for i in range(3, int(n**(0.5)) + 1, 2):
        if n % i == 0:
            return False
        
    return True

In [3]:
# Buscando os primos de 10.000 até
# 19.999

if 'primos.npy' not in glob('*.npy'):
    primes = []
    a = 10000
    for i in range(5000):
        if is_prime(a + 2 * i + 1):
            primes.append(a + 2 * i + 1)

    primes = np.array(primes)
    with open('primos.npy', 'wb') as f:
        np.save(f, primos)

else:
    primes = np.load('primos.npy')

primes

array([10007, 10009, 10037, ..., 19991, 19993, 19997])

In [4]:
def choose_parameters(primes):
    '''
    Recebe um array de primos e
    retorna dois primos p e q,
    phi(n) e números e e d de
    modo que de = 1 (mod phi(n)),
    onde n = p*q
    '''
    
    rd.shuffle(primes)
    p, q = primes[:2]
    phi_n = (p - 1) * (q - 1)
    
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
    
    return p, q, phi_n, e, d

def str2int(message):
    exp = 1
    number = 0
    message = message[::-1]
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    message = message[::-1]
    return message

def generate_keys(p, q):
    phi_n = (p - 1) * (q - 1)
    e = 2
    lcd, _, d = euclid_extended(phi_n, e)
    while lcd != 1:
        e = rd.randint(3, phi_n)
        lcd, _, d = euclid_extended(phi_n, e)
        
    return (phi_n, e), (phi_n, d)

def encrypt(message, public_key):
    phi_n, e = public_key
    phi_n = int(phi_n)
    m = str2int(message)
    m = (m * e) % phi_n
    encrypted = int2str(m)
    
    return encrypted

def decrypt(encrypted, private_key):
    phi_n, d = private_key
    d = int(d)
    phi_n = int(phi_n)
    m = str2int(encrypted)
    m = (m * d) % phi_n
    message = int2str(m)
    
    return message

In [5]:
primes = [100000000000000003, 100000000000000013]

# alguns primos de https://en.wikipedia.org/wiki/Largest_known_prime_number
primes = primes + [999999000001, 67280421310721, 170141183460469231731687303715884105727, 20988936657440586486151264256610222593863921, 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127, 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151, 10407932194664399081925240327364085538615262247266704805319112350403608059673360298012239441732324184842421613954281007791383566248323464908139906605677320762924129509389220345773183349661583550472959420547689811211693677147548478866962501384438260291732348885311160828538416585028255604666224831890918801847068222203140521026698435488732958028878050869736186900714720710555703168729087]

In [6]:
p, q = primes[-1], primes[-2]

if 'public.csv' not in glob('*.csv'):
    public_key, private_key = generate_keys(p, q)
    with open('public.csv', 'w') as file:
        file.write(str(public_key[0]) + '\n')
        file.write(str(public_key[1]) + '\n')
        
    with open('private.csv', 'w') as file:
        file.write(str(private_key[0]) + '\n')
        file.write(str(private_key[1]) + '\n')   
else:
    file = open('public.csv')
    public_key = file.readlines()
    file.close()
    public_key = tuple([int(i) for i in public_key])
    
    file = open('private.csv')
    private_key = file.readlines()
    file.close()
    private_key = tuple([int(i) for i in private_key])

In [7]:
import smtplib
from getpass import getpass
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_mail(from_mail, to_mail, subject, body, public_key, tag = '[Encrypted] '):
    password = getpass(prompt = 'Password: ')
    clear_output()
    body = encrypt(body, public_key)
    subject = tag + subject
    msg = MIMEMultipart()
    msg['From'] = from_mail
    msg['To'] = to_mail
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls()
    server.ehlo()
    server.login(from_mail, password)
    text = msg.as_string()
    server.sendmail(from_mail, to_mail, text)
    server.quit()

In [8]:
from_mail = 'igorpmichels@gmail.com'
to_mail = 'luca.escopelli@gmail.com'
subject = 'Eclass fora do ar?'
body = 'O eclass ta fora do ar tbm? sei q zap, face e insta tão fora, mas tbm não consigo entrar no eclass. Como q vou estudar algebra? como q vou fazer a lista pra amanha?'
send_mail(from_mail, to_mail, subject, body, public_key)

In [9]:
import smtplib
import time
import imaplib
import email
import traceback 
from email.header import decode_header

def read_email_from_gmail(username, password, server, private_key, tag = '[Encrypted]', qtd = 1):
    printed = 0
    mail = imaplib.IMAP4_SSL(server)
    mail.login(username, password)
    mail.select('inbox')
    data = mail.search(None, 'ALL')
    mail_ids = data[1]
    id_list = mail_ids[0].split()   
    first_email_id = int(id_list[0])
    latest_email_id = int(id_list[-1])
    for i in range(latest_email_id, first_email_id, -1):
        if printed == qtd:
            break

        data = mail.fetch(str(i), '(RFC822)')
        for response_part in data:
            arr = response_part[0]
            if isinstance(arr, tuple):
                msg = email.message_from_string(str(arr[1], 'utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                if tag not in email_subject:
                    break
                else:
                    printed += 1

                # email_subject = decode_header(email_subject)[0][0].decode('utf-8')
                # print(email_subject)
                # email_subject = decrypt(email_subject[12:], private_key)
                print('From: ' + email_from)
                print('Subject: ' + email_subject)
                print()
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get('Content-Disposition'))
                    try:
                        body = part.get_payload(decode = True).decode()
                    except:
                        pass

                    if content_type == 'text/plain':
                        body = decrypt(body, private_key)
                        print(body)
        
        print()

# username = 'igorpmichels@gmail.com'
# password = getpass(prompt = 'Password: ')
# clear_output()
# server = 'imap.gmail.com'
# read_email_from_gmail(username, password, server, private_key, tag = '[Encrypted]')